In [1]:
!pip install pandas plotly kaleido

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import os
import re
import plotly.express as px 
import pandas as pd 
import numpy as np 

In [3]:
def load_data(path):
    dfs = []

    for x in os.listdir(path):
        procs, sleep = re.match(r"result_(.*)_1000_(.*).csv", x).groups()
        procs, sleep = int(procs), float(sleep)
        df = pd.read_csv(path + "/" + x)
        df['id'] = df.index
        df['procs'] = procs
        df['sleep'] = sleep
        dfs.append(df)
    df = pd.concat(dfs)
    df["time_in"] = pd.to_datetime(df['time_in'])
    df["time_mid"] = pd.to_datetime(df['time_mid'])
    df["time_end"] = pd.to_datetime(df['time_end'])

    df["diff_in_end"] = (df['time_end'] - df['time_in']).apply(lambda x: x.microseconds / 1000)
    df["diff_in_mid"] = (df['time_mid'] - df['time_in']).apply(lambda x: x.microseconds / 1000)
    df["diff_mid_end"] = (df['time_end'] - df['time_mid']).apply(lambda x: x.microseconds / 1000)

    return df

In [4]:
dfp = load_data("./postgres/results/")
dfp["db"] = "postgres"

In [5]:
dfr = load_data("./redis/results/")
dfr["db"] = "redis"

In [6]:
df = pd.concat([dfr, dfp]).drop("id", axis=1)

In [7]:
df.to_csv("data.csv", index=False)

In [8]:
labels={"db": "Database", "diff_in_end": "proc1 to proc3 latency (ms)", "diff_mid_end": "proc2 to proc3 latency (ms)", "diff_in_mid": "proc1 to proc2 latency (ms)", "sleep": "sleep proc1 (s)", "procs": "processes", "emmit_type": "process 3 decision"}

In [9]:
df_mean = df.groupby(["procs", "sleep", "db"]).mean().reset_index()
df_mean

,procs,sleep,db,emmit_type,diff_in_end,diff_in_mid,diff_mid_end
0,1,0.001,postgres,0.725000,9.530679,4.639434,11.891245
1,1,0.001,redis,1.169000,0.949251,0.543142,0.406109
2,1,0.003,postgres,0.781000,6.957875,4.152586,4.805289
3,1,0.003,redis,1.179000,2.234268,1.302853,2.931415
4,1,0.006,postgres,0.754000,5.322617,3.406457,6.916160
...,...,...,...,...,...,...,...
123,16,0.003,redis,1.204750,439.873933,18.047887,436.451046
124,16,0.006,postgres,0.773813,30.694044,21.825921,249.243123
125,16,0.006,redis,1.145563,5.577283,1.399559,19.052724
126,16,0.009,postgres,0.767125,21.615943,15.541687,241.511756


In [10]:
df_succ = (df[df["diff_in_end"] <= 20].groupby(["db", "procs", "sleep"])["diff_in_end"].count() / df.groupby(["db", "procs", "sleep"])["diff_in_end"].count()).reset_index()
df_succ = df_succ.rename(columns={"diff_in_end": "success rate"})
df_succ

,db,procs,sleep,success rate
0,postgres,1,0.001,0.979000
1,postgres,1,0.003,0.980000
2,postgres,1,0.006,0.997000
3,postgres,1,0.009,0.998000
4,postgres,2,0.001,0.766000
...,...,...,...,...
123,redis,15,0.009,0.994867
124,redis,16,0.001,0.021500
125,redis,16,0.003,0.051437
126,redis,16,0.006,0.944625


In [11]:
df_emmit_type = df.groupby(["procs", "sleep", "db", "emmit_type"]).mean().reset_index()
df_emmit_type

,procs,sleep,db,emmit_type,diff_in_end,diff_in_mid,diff_mid_end
0,1,0.001,postgres,0.0,8.931624,4.582914,17.784411
1,1,0.001,postgres,1.0,10.348124,4.878884,5.469240
2,1,0.001,postgres,2.0,10.025163,4.532341,5.492821
3,1,0.001,redis,0.0,0.696953,0.524264,0.172689
4,1,0.001,redis,1.0,0.988446,0.548712,0.439733
...,...,...,...,...,...,...,...
379,16,0.009,postgres,1.0,27.627872,15.621163,12.006709
380,16,0.009,postgres,2.0,27.452652,15.432286,12.020366
381,16,0.009,redis,0.0,1.104364,0.659731,53.361050
382,16,0.009,redis,1.0,1.408142,0.669849,0.738293


In [19]:
fig = px.line(df_mean[(df_mean["db"] == "redis")], x='procs', y='diff_in_end', color="sleep", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_redis_all_sleeps.png", scale=5, engine="kaleido")
fig.show()
fig = px.line(df_mean[(df_mean["db"] == "postgres")], x='procs', y='diff_in_end', color="sleep", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_postgres_all_sleeps.png", scale=5, engine="kaleido")
fig.show()

In [20]:
fig = px.line(df_mean[df_mean["sleep"] == 0.001], x='procs', y='diff_in_end', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_postgres_vs_redis0001.png", scale=5, engine="kaleido")
fig.show()
fig = px.line(df_mean[df_mean["sleep"] == 0.003], x='procs', y='diff_in_end', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_postgres_vs_redis0003.png", scale=5, engine="kaleido")
fig.show()
fig = px.line(df_mean[df_mean["sleep"] == 0.006], x='procs', y='diff_in_end', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_postgres_vs_redis0006.png", scale=5, engine="kaleido")
fig.show()
fig = px.line(df_mean[df_mean["sleep"] == 0.009], x='procs', y='diff_in_end', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_postgres_vs_redis0009.png", scale=5, engine="kaleido")
fig.show()

In [21]:
fig = px.line(df_succ[(df_succ["db"] == "redis")], x='procs', y='success rate', color="sleep", labels=labels)
fig.write_image("./graphs/success_rate_redis_all_sleeps.png", scale=5, engine="kaleido")
fig.show()

fig = px.line(df_succ[(df_succ["db"] == "postgres")], x='procs', y='success rate', color="sleep", labels=labels)
fig.write_image("./graphs/success_rate_postgres_all_sleeps.png", scale=5, engine="kaleido")
fig.show()

In [22]:
fig = px.line(df_succ[(df_succ["sleep"] == 0.001)], x='procs', y='success rate', color="db", labels=labels)
fig.write_image("./graphs/success_rate_postgres_vs_redis_0001.png", scale=5, engine="kaleido")
fig.show()

fig = px.line(df_succ[(df_succ["sleep"] == 0.003)], x='procs', y='success rate', color="db", labels=labels)
fig.write_image("./graphs/success_rate_postgres_vs_redis_0003.png", scale=5, engine="kaleido")
fig.show()

fig = px.line(df_succ[(df_succ["sleep"] == 0.006)], x='procs', y='success rate', color="db", labels=labels)
fig.write_image("./graphs/success_rate_postgres_vs_redis_0006.png", scale=5, engine="kaleido")
fig.show()

fig = px.line(df_succ[(df_succ["sleep"] == 0.009)], x='procs', y='success rate', color="db", labels=labels)
fig.write_image("./graphs/success_rate_postgres_vs_redis_0009.png", scale=5, engine="kaleido")
fig.show()

In [23]:
fig = px.line(df_mean[df_mean["sleep"] == 0.003], x='procs', y='diff_in_end', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_end_postgres_vs_redis.png", scale=5, engine="kaleido")
fig.show()
fig = px.line(df_mean[df_mean["sleep"] == 0.003], x='procs', y='diff_in_mid', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_in_mid_postgres_vs_redis.png", scale=5, engine="kaleido")
fig.show()
fig = px.line(df_mean[df_mean["sleep"] == 0.003], x='procs', y='diff_mid_end', color="db", log_y=True, labels=labels)
fig.write_image("./graphs/diff_mid_end_postgres_vs_redis.png", scale=5, engine="kaleido")
fig.show()

In [24]:
fig = px.line(df_emmit_type[(df_emmit_type["sleep"] == 0.003) & (df_emmit_type["db"] == "postgres")], x='procs', y='diff_in_end', color="emmit_type", log_y=True, labels=labels)
fig.write_image("./graphs/proc_type_postgres.png", scale=5, engine="kaleido")
fig.show()

In [25]:
fig = px.line(df_emmit_type[(df_emmit_type["sleep"] == 0.003) & (df_emmit_type["db"] == "redis")], x='procs', y='diff_in_end', color="emmit_type", log_y=True, labels=labels)
fig.write_image("./graphs/proc_type_redis.png", scale=5, engine="kaleido")h
fig.show()